# Design Codes
This example demonstrates how to work with design codes. We start by importing the necessary modules.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from shapely.geometry import Polygon
import shapely.geometry
from concreteproperties.design_codes import NZS3101
import concreteproperties.stress_strain_profile as ssp
from sectionproperties.pre.library.concrete_sections import concrete_rectangular_section
from concreteproperties.concrete_section import ConcreteSection
from concreteproperties.results import MomentInteractionResults
from concreteproperties.results import BiaxialBendingResults
from concreteproperties.results import MomentCurvatureResults

## Create Design Code and Materials
In this example we'll use the NZS3101:2006 design code. 40 MPa concrete with default density of 2300 kg/m^3 will be used with grade 300E reinforcing steel.

In [ ]:
design_code = NZS3101()
concrete_40 = design_code.create_concrete_material(compressive_strength=40)
steel_300E = design_code.create_steel_material(steel_grade='300E')

We can confirm the concrete material properties.

In [ ]:
print(concrete_40.name)
concrete_40.stress_strain_profile.plot_stress_strain(title="Concrete Serviceability Stress-Strain Profile")
concrete_40.ultimate_stress_strain_profile.plot_stress_strain(title="Concrete Ultimate Stress-Strain Profile")
print(
    f"Concrete Flexural Tensile Strength: {concrete_40.flexural_tensile_strength:.2f} MPa"
)

We can confirm the steel material properties.

In [ ]:
print(steel_300E.name)
print(f"Density = {steel_300E.density} kg/m^3")
steel_300E.stress_strain_profile.plot_stress_strain(title="Steel Stress-Strain Profile")

## Assign Geometry to Design Code
This example will analyse a 800D x 500W concrete beam with 6-D25 top and 6-D20 bottom bars with an assumed R12 stirrup size with a specified 35 concrete cover all round. It is assumed the analysis of the concrete frame containing the beam in question is based on a maximum global ductility demand of $\mu$ = 3. An assessment of the Potential Plastic Hinge Region (PPHR) curvature has shown that the PPHR can be designated as a Limited Ductile Plastic Region (LDPR). After creating the geometry it must be assigned to the design code object.

In [ ]:
geom_beam = concrete_rectangular_section(
    b=500,
    d=800,
    dia_top=25,
    n_top=5,
    dia_bot=20,
    n_bot=5,
    n_circle=8,
    cover=35+12,
    area_top=25**2*np.pi/4,
    area_bot=20**2*np.pi/4,
    conc_mat=concrete_40,
    steel_mat=steel_300E,
)

conc_sec_beam = ConcreteSection(geom_beam)
conc_sec_beam.plot_section()
design_code.assign_concrete_section(concrete_section=conc_sec_beam)

## Area Properties
Obtaining the area properties is identical to that of a ``ConcreteSection`` object. The same can be done for a moment-curvature analysis and stress analyses (not carried out in this example).

In [ ]:
gross_props = design_code.get_gross_properties()
transformed_props = design_code.get_transformed_gross_properties(
    elastic_modulus=concrete_40.stress_strain_profile.elastic_modulus
)
cracked_props = design_code.calculate_cracked_properties()

## Ultimate Bending Capacity
The factored ultimate bending capacity can be found by calling the ``ultimate_bending_capacity()`` method. This method returns a factored and unfactored ``UltimateBendingResults`` object, as well as the capacity reduction factor ``phi``.

The NZS3101 design code includes five valid analysis types that can be provided to the analysis method:-
- ``nom_chk``
- ``cpe_chk``
- ``os_chk``
- ``prob_chk``
- ``prob_os_chk``

Based on our analysis of the frame our column face beam moment demands are $M^*_{pos}$ = 300 kNm, and $M^*_{neg}$ = 450 kNm, the column is an exterior column with a single beam framing into it on one face.

Therefore we need to do a nominal design check analysis for both positive and negative flexure to compare with the $\mu$ = 3 demands and match the flexural strength provided as closely as possible to the demands:-

In [ ]:
M_star_pos = 280.0
M_star_neg = -442.0

f_ult_res_pos, ult_res_pos, phi_pos = design_code.ultimate_bending_capacity(theta=0,pphr_class="LDPR",analysis_type="nom_chk")
print("Positive flexure capacity:-")
print(f"M_star_pos = {M_star_pos} kN.m")
print(f"Mn_pos = {ult_res_pos.m_x / 1e6:.2f} kN.m")
print(f"phi = {phi_pos:.3f}")
print(f"phi.Mn_pos = {f_ult_res_pos.m_x / 1e6:.2f} kN.m")
dc_ratio_pos = M_star_pos*1e6 / f_ult_res_pos.m_x
print(f"Demand/Capacity ratio = {abs(dc_ratio_pos):.3f}")
print()

f_ult_res_neg, ult_res_neg, phi_neg = design_code.ultimate_bending_capacity(theta=np.pi,pphr_class="LDPR",analysis_type="nom_chk",)
print("Negative flexure capacity:-")
print(f"M_star_neg = {M_star_neg} kN.m")
print(f"Mn_neg = {ult_res_neg.m_x / 1e6:.2f} kN.m")
print(f"phi = {phi_neg:.3f}")
print(f"phi.Mn_neg = {f_ult_res_neg.m_x / 1e6:.2f} kN.m")
dc_ratio_neg = M_star_neg*1e6/f_ult_res_neg.m_x
print(f"Demand/Capacity ratio = {abs(dc_ratio_neg):.3f}")

We can see the nominal design capacity is a close match to the ductile design demands (as ratio is close to 1.000), so the proposed reinforcement is deemed adequate for the proposed beam size without being overly conservative which would otherwise unnecessarily push up our overstrength demands. 

Because we are undertaking a ductile design we must now determine the overstrength capacity of the beam section so we can undertake the neccessary capacity design checks on the columns supporting the beam. We undertake this check using an ``analysis_type`` of ``os_chk``, this internally factors up the concrete and steel materials to reflect the likely upper limit strengths (overstrength material properties) before undertking the cross section analysis.

For an analysis to NZS3101:2006, this typically means the concrete strength increase is based on $f'_c$ + 15 MPa, and steel strength increase is based on of $\phi_{o,f_y}f_y$, where typically $\phi_{o,f_y}$ = 1.35 for grade 300E & 500E steel reinforcement materials that complies with AS/NZS4671. This is intended to capture the maximum possible actual value of the beam strengths to ensure that the columns have sufficient strength for based on capacity design principles. Note a $\phi$ of 1.0 is utilised for this check in accordance with NZS3101:2006.

In [ ]:
f_ult_res_pos_os, _, _ = design_code.ultimate_bending_capacity(theta=0,pphr_class="LDPR",analysis_type="os_chk")
print("Positive overstrength demand:-")
print(f"phi_o.Mo = {f_ult_res_pos_os.m_x / 1e6:.2f} kN.m")
os_ratio_pos = f_ult_res_pos_os.m_x/(M_star_pos*1e6)
print(f"Actual positive flexure overstrength ratio = {abs(os_ratio_pos):.3f}")
print()

f_ult_res_neg_os, _, _ = design_code.ultimate_bending_capacity(theta=np.pi,pphr_class="LDPR",analysis_type="os_chk")
print("Negative overstrength demand:-")
print(f"phi_o.Mo = {f_ult_res_neg_os.m_x / 1e6:.2f} kN.m")
os_ratio_neg = f_ult_res_neg_os.m_x/(M_star_neg*1e6)
print(f"Actual negative flexure overstrength ratio = {abs(os_ratio_neg):.3f}")

This shows that our original nominal analysis moments based on $\mu$ = 3 actions are factored up by a 1.675 & 1.615 ratio for positive and negative flexure respectively for determining the overstrength beam moments applied at the face of the supporting columns.

Now that we have our beam overstrength demand, we can check a proposed column design. Note in this example we will assume half of the beam overstrength demand as a moment is distributed to the column above and below the beam column joint. In an actual design this ratio would be determined from the analysis, and may require to be factered up further due to dynamic amplification factors. Note as well in a real capacity design scenario the face beam overstrength moments would need to be converted to centreline moments before distributing to the columns to determine the column face moments at top and bottom of the beam/column joint. But for simplicity here to demonstrate a column analysis we will make the simplification of distributing the beam overstrenght moment evenly above and below the beam/column joint.

Firstly we must define our column section

This example will analyse a 600D x 600W concrete column with 12-DH20 bars evenly distributed around the perimeter with an assumed R12 stirrup size with a specified 35 concrete cover all round. Since the columns in this scenario are a Capacity Protected Element (CPE) an ``analysis_type`` of ``cpe_chk`` can be utilised. This means the analysis is based on normal characteristic strengths, but a $\phi$ = 1.0 can be utilised since the demands being checked were derived on a capacity design basis (i.e. considering overstrength effects). 

An assessment of the curvatures in the PPHR's of the columns indicate that they would be within the Nominally Ductile Plastic Region (NDPR) curvature limits, note that this is the default PPHR region classification so does not need to be specifically entered, but is provided here for completeness when undertaking the analysis.

Under the negative flexure case the column has an axial tension load of $N_{o,T}$ of -165 kN under the joint, and under the positive flexure case the column has an axial compression overstrength derived load of $N_{o,C}$ = 1100 kN under the joint.

Based on the assumption of half of the beam overstrength moment distributing to the column above and below the beam/column joint this lead to a design moment for the column combined with the tension load of $M_{o,T}$ = -713.9/2 = -356.95 kNm, and combined with the overstrength compression load of $M_{o,C}$ = 469.0/2 = 234.5 kNm.  

Since we are utlising higher strength grade 500E reinforcment in the column, we must first define a new steel material. Note that the same 40 MPa concrete material is being used in the columns.


In [ ]:
steel_500E = design_code.create_steel_material(steel_grade='500E')

We can now create the column section and assign it to the design code, and plot the section to check the geometry

In [ ]:
geom_col = concrete_rectangular_section(
    b=600,
    d=600,
    dia_top=20,
    n_top=4,
    dia_bot=20,
    n_bot=4,
    dia_side=20,
    n_side=2,
    n_circle=8,
    cover=35+12,
    area_top=20**2*np.pi/4,
    area_bot=20**2*np.pi/4,
    area_side=20**2*np.pi/4,
    conc_mat=concrete_40,
    steel_mat=steel_500E,
)

conc_sec_col = ConcreteSection(geom_col)
conc_sec_col.plot_section()
design_code.assign_concrete_section(concrete_section=conc_sec_col)

We can now analyse the section to determine at the axial load required to be supported what the actual moment capacity. Since the section is symmetrical, we can just look at the positive result, but if the opposite bending direction was required this can be determined by utilising the ``theta`` parameter to rotate the neutral axis 180 $\!^{\circ}$ or $\pi$ rads for the analysis.

In [ ]:
M_o_T = -713.91/2
N_o_T = -165
M_o_C = 468.97/2
N_o_C = 1100

f_ult_res_T, _, _ = design_code.ultimate_bending_capacity(pphr_class="NDPR",analysis_type="cpe_chk", n=N_o_T*1e3)
print("Tension case capacity:-")
print(f"phi.Mn = {f_ult_res_T.m_x / 1e6:.2f} kN.m at axial load of {N_o_T:.2f} kN")
print(f"Mo = {M_o_T} kN.m")
col_ratio_T = M_o_T*1e6/f_ult_res_T.m_x
print(f"Actual design ratio = {abs(col_ratio_T):.3f}")
print()

f_ult_res_C, _, _ = design_code.ultimate_bending_capacity(pphr_class="NDPR",analysis_type="cpe_chk", n=N_o_C*1e3)
print("Compression case capacity:-")
print(f"phi.Mn = {f_ult_res_C.m_x / 1e6:.2f} kN.m at axial load of {N_o_C:.2f} kN")
print(f"Mo = {M_o_C} kN.m")
col_ratio_C = M_o_C*1e6/f_ult_res_C.m_x
print(f"Actual design ratio = {abs(col_ratio_C):.3f}")


Therefore we can see that the flexural design would be sufficient for the proposed column.

We can also plot these points on a M/N interaction diagrams and a biaxial Mx/My interaction diagram to show that the target axial and moments that the design lies within the interaction surface.

## Axial/Moment Interaction Diagram
We can also plot these points on M/N interaction diagrams and Mx/My interaction diagrams to better show that the target axial and moments actions for the design are within the interaction surface.

In [ ]:
f_mi_res, mi_res, phis = design_code.moment_interaction_diagram(pphr_class="NDPR",analysis_type="cpe_chk")

We can compare the factored and unfactored moment interaction diagrams. Since $\phi$ = 1.0 the factored and unfactors interaction curves should coincide.

In [ ]:
MomentInteractionResults.plot_multiple_diagrams(
    [f_mi_res, mi_res], ["Factored", "Unfactored"], fmt="-"
)

We can check to see if a combination of axial force and bending moment lies within the moment interaction diagram using the ``point_in_diagram()`` method.

In [ ]:
# design load cases
n_stars = [N_o_T*1e3, N_o_C*1e3]
m_stars = [-M_o_T*1e6, M_o_C*1e6]
marker_styles = ["s", "o"]
n_cases = len(n_stars)

# plot moment interaction diagram
ax = f_mi_res.plot_diagram(fmt="k-", render=False)

# check to see if combination is within diagram and plot result
for idx in range(n_cases):
    case = f_mi_res.point_in_diagram(n=n_stars[idx], m=m_stars[idx])
    print("Case {num}: {status}".format(num=idx + 1, status="OK" if case else "FAIL"))
    ax.plot(
        m_stars[idx] / 1e6,
        n_stars[idx] / 1e3,
        "k" + marker_styles[idx],
        markersize=5,
        label=f"Case {idx + 1}",
    )

ax.legend()
plt.show()

## Biaxial Bending Diagram
We can also compute factored biaxial bending diagrams by calling the ``biaxial_bending_diagram()`` method. This method returns a factored ``BiaxialBendingResults`` object as well as a list of the capacity reduction factors ``phis``.

In [ ]:
# create biaxial bending diagram
f_bb_res1, phis1 = design_code.biaxial_bending_diagram(n=N_o_T*1e3)
f_bb_res2, phis2 = design_code.biaxial_bending_diagram(n=N_o_C*1e3)


In [ ]:
# plot case 1
ax = f_bb_res1.plot_diagram(fmt="-k", render=False)
ax.plot(M_o_T,0,"sk")
plt.show()

# plot case 2
ax = f_bb_res2.plot_diagram(fmt="-k", render=False)
ax.plot(M_o_C,0,"ok")
plt.show()


In [ ]:
# determine maximum compression load for the column section
max_comp = design_code.max_comp_strength(cpe_design=True)
max_ten = design_code.max_ten_strength()
steps = 15
#generate axial load list 
n_list = np.linspace(-max_ten+20000, max_comp, steps)
theta_list = np.linspace(0.0,2*np.pi,48,False)

results_bb = []
results_mi = []
labels = []

for theta in theta_list:
    f_mi_res, _, _ = design_code.moment_interaction_diagram(pphr_class="NDPR", analysis_type="cpe_chk", theta=theta)
    results_mi.append(f_mi_res)
for n in n_list:
    f_bb_res, _ = design_code.biaxial_bending_diagram(pphr_class="NDPR", analysis_type="cpe_chk", n=n)
    results_bb.append(f_bb_res)
    labels.append(f"N* = {n/1e3}")
    
# plot all the diagrams on one image
fig = plt.figure(figsize=(12,10))
ax = plt.axes(projection="3d")
n_scale = 1e-3
m_scale = 1e-6
for mi_res in results_mi:
    n, m_x_list = mi_res.get_results_lists(moment='m_x')
    _, m_y_list = mi_res.get_results_lists(moment='m_y')
    n = np.array(n) * n_scale
    m_x_list = np.array(m_x_list) * m_scale
    m_y_list = np.array(m_y_list) * m_scale
    
    ax.plot3D(m_x_list, m_y_list, n, "-k", linewidth=0.5)

for i, bb_res in enumerate(results_bb):
    m_x_list, m_y_list = bb_res.get_results_lists()
    m_x_list = np.array(m_x_list) * m_scale
    m_y_list = np.array(m_y_list) * m_scale

    n = np.array(n_list[i]) * n_scale
    ax.plot3D(m_x_list, m_y_list, n, "-r", linewidth=0.5)

ax.set_xlabel("Bending Moment $M_x$")
ax.set_ylabel("Bending Moment $M_y$")
ax.set_zlabel("Axial Force $N$")

plt.show()

## Probable Strength Analyses
The ``NZS3101`` design code also allows for the analysis of concrete cross sections in accordance with the NZSEE C5 Assessment Guidelines.

These guidelines are intended for the assesment of existing buildings based on probable material strength assumptions for the steel and concrete materials. Probable strengths of material are intended to represent more of an average strength approach, recognising that in the real world the material strengths are less likely to all be a 5% characteristic strength.

To utilise these types of analyses within ``ultimate_bending_capacity()``, ``moment_interaction_diagram()`` and ``biaxial_bending_diagram()`` methods we can specify an ``analysis_type`` of ``prob_chk`` or ``prob_os_chk`` to undertake a normal or overstrength probable strength based analysis. We can specify predefined probable strength steel materials that are based on common historic material probable strength properties from the NZSEE C5 guidelines, or we can specfy a steel material based on characteristic properties and the analysis will internally scale these to an appropriate probable strength based material. Similarly concrete strengths are specified on the basis of characteristic strengths and are internally scaled to the appropriate probable strength based concrete material.

Note we will undertake a moment curvature analysis based on an unconfined analysis to NZSEE C5 asseement guidelines

The section we will analysis will be a 600D x 600W column with 8-D28 grade 275 bars, and D10 stirrups with 30 cover and an original specified characteristic concrete strength of 25 MPa.

So we utilise these parameters to create a new concrete cross section with a predefined steel material with probable strength properties and the characteristic concrete strength properties. 

Except note that that for a moment curvature analysis to NZSEE C5 assessment guidelines we are able to utilise a higher ultimate concrete strain of $\varepsilon_{c,max}$ = 0.004 for an unconfined analysis. Note also for a moment-curvature analysis to the NZSEE C5 assessment guidelines that the limiting strain in the reinforcement is taken as $\varepsilon_{c,max}$ = 0.6 $\!\varepsilon_{cu}$ but $\leq$ 0.06 (or 6% strain). Note if the section is able to be classified as being confined, the concrete limiting strain can be even higher leading to increased ductility of the cross section (refer to NZSEE C5 assessment guidelines further further discussion).

In [ ]:
#create new materials overriding default ultimate strains with values appropriate for the moment-curvature analysis
steel_275 = design_code.create_steel_material(steel_grade='275',fracture_strain=min(0.06,15/100*0.6))
concrete_25 = design_code.create_concrete_material(compressive_strength=25,ultimate_strain=0.004)

We can show that the predefined material has a probable yield strength of 324 MPa.

In [ ]:
print(steel_275.name)
print(f"Density = {steel_275.density} kg/m^3")
print(f"Probable yield strength = {steel_275.stress_strain_profile.get_yield_strength()} MPa")
steel_275.stress_strain_profile.plot_stress_strain(title="Steel Stress-Strain Profile")

Next we create our column section and assign it to the design code.

In [ ]:
geom_col = concrete_rectangular_section(
    b=600,
    d=600,
    dia_top=28,
    n_top=3,
    dia_bot=28,
    n_bot=3,
    dia_side=28,
    n_side=1,
    n_circle=8,
    cover=30+10,
    area_top=28**2*np.pi/4,
    area_bot=28**2*np.pi/4,
    area_side=28**2*np.pi/4,
    conc_mat=concrete_25,
    steel_mat=steel_275,
)

conc_sec = ConcreteSection(geom_col)
conc_sec.plot_section()
design_code.assign_concrete_section(concrete_section=conc_sec)

You will note that when using a predefined probable strength based steel grade/material that the reinforcing bars are plotted with a blue colour by default to distinguish them from a characteristic strength based material.

Next we will need to create a section with probable strength properties from the previously defined section, and reassign it to the design code for the moment-curvature analysis. 
 

In [ ]:
prob_strength_sec = design_code.create_prob_section()
design_code.assign_concrete_section(concrete_section=prob_strength_sec)


We can show now that the concrete strength has been internally scaled to the probable strength of 1.5 x 25 = 37.5 MPa

In [ ]:
print(f"Probable concrete yield strength = {design_code.concrete_section.concrete_geometries[0].material.ultimate_stress_strain_profile.get_compressive_strength()} MPa")

Given we now have the probable strength based section assigned we can undertake the moment curvature analysis, demonstrating that under higher axial loads that the section is able to achieve a higher curvature.

In [ ]:
moment_curvature_results = design_code.moment_curvature_analysis()
print(f"Failure material is:-\n{moment_curvature_results.failure_geometry.material.name}")

MomentCurvatureResults.plot_results(moment_curvature_results,fmt="-k")

We can determine from the moment-curvature analysis the maximum moment capacity:- NEED TO UPDATE THIS TO ACTUALLY WORK OUT THE YIELD CAPACITY

In [ ]:
max_moment = max(moment_curvature_results.m_x)*1e-6
print(f"max_moment = {max_moment:.2f} kN.m")

We can assess the same sections probable strength. Note we must recreate the materials and hence section due to the previously specified custom limiting strains for the moment-curvature analysis.

In [ ]:
concrete_25 = design_code.create_concrete_material(compressive_strength=25)
steel_275 = design_code.create_steel_material(steel_grade='275')

geom_col = concrete_rectangular_section(
    b=600,
    d=600,
    dia_top=28,
    n_top=3,
    dia_bot=28,
    n_bot=3,
    dia_side=28,
    n_side=1,
    n_circle=8,
    cover=30+10,
    area_top=28**2*np.pi/4,
    area_bot=28**2*np.pi/4,
    area_side=28**2*np.pi/4,
    conc_mat=concrete_25,
    steel_mat=steel_275,
)

conc_sec = ConcreteSection(geom_col)
conc_sec.plot_section()
design_code.assign_concrete_section(concrete_section=conc_sec)

In [ ]:

n = 0*1e3
f_ult_res, _, _ = design_code.ultimate_bending_capacity(pphr_class="NDPR",analysis_type="prob_chk", n=n)

print("Probable moment capacity:-")
print(f"Mp = {f_ult_res.m_x / 1e6:.2f} kN.m at axial load of {n/1e3:.2f} kN")

This shows good agreement with the moment-curvature analysis.